# Lesson 4 Assignment

In this lab assignment, you will build a non-tree-based classifier where you can ensemble any base-learners and pass it to the using a BaggingClassifier or any of the other ensemble learners in sklearn.ensemble.

In [1]:
# import packages
%matplotlib inline
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns
import pandas as pd
from sklearn.datasets import make_moons
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.utils import resample

# make this notebook's output stable across runs
np.random.seed(0)

Matplotlib is building the font cache; this may take a moment.


## Data Set Information:

This dataset represents a set of possible advertisements on Internet pages. The features encode the geometry of the image (if available) as well as phrases occuring in the URL, the image's URL and alt text, the anchor text, and words occuring near the anchor text. The task is to predict whether an image is an advertisement ("ad") or not ("nonad"). Additional information can be found [here](https://archive.ics.uci.edu/ml/datasets/internet%2Badvertisements).

## Attribute Information:

The dataset has 3 continous (height, width, aratio) and 1555 binary (urls, tags, captions) features. 

## Source:

Creator & donor: Nicholas Kushmerick <nick '@' ucd.ie>

In [4]:
# Load the data
df = pd.read_csv('Internet_Ad_Data.csv', sep=',')
print(internetAd.info())
df.head(20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3279 entries, 0 to 3278
Columns: 1559 entries, height to Target
dtypes: int64(1554), object(5)
memory usage: 39.0+ MB
None


/usr/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3319: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,height,width,aratio,local,url*images+buttons,url*likesbooks.com,url*www.slake.com,url*hydrogeologist,url*oso,url*media,...,caption*home,caption*my,caption*your,caption*in,caption*bytes,caption*here,caption*click,caption*for,caption*you,Target
0,125,125,1.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ad.
1,57,468,8.2105,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ad.
2,33,230,6.9696,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ad.
3,60,468,7.8,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ad.
4,60,468,7.8,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ad.
5,60,468,7.8,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ad.
6,59,460,7.7966,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ad.
7,60,234,3.9,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ad.
8,60,468,7.8,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ad.
9,60,468,7.8,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ad.


Question 1: Prepare and impute missing values with the median

In [5]:
missing_val_columns = ['height', 'width', 'aratio', 'local']
for column in missing_val_columns:
    median = df[~df[column].astype(str).str.contains("\?")][column].median()
    df[column] = df[column].replace("\?", median, regex=True)
df.head()

,height,width,aratio,local,url*images+buttons,url*likesbooks.com,url*www.slake.com,url*hydrogeologist,url*oso,url*media,...,caption*home,caption*my,caption*your,caption*in,caption*bytes,caption*here,caption*click,caption*for,caption*you,Target
0,125,125,1.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ad.
1,57,468,8.2105,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ad.
2,33,230,6.9696,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ad.
3,60,468,7.8,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ad.
4,60,468,7.8,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ad.


Question 2: Split dataset into training and test set

In [6]:
from sklearn.model_selection import train_test_split

X = df.drop('Target', axis=1)
y = df['Target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=99)

Question 3: Train and evaluate a LogisticRegression classifier using LogisticRegression.Use Gridsearch CV to determine the best k.

In [19]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(max_iter=500, solver='liblinear')
params = {
    'C': [.1, 1, 10],
    'penalty': ['l1', 'l2']
}
clf_grid = GridSearchCV(clf, param_grid=params, cv=5)
clf_grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=LogisticRegression(max_iter=500, solver='liblinear'),
             param_grid={'C': [0.1, 1, 10], 'penalty': ['l1', 'l2']})

In [20]:
clf_grid.best_params_

{'C': 10, 'penalty': 'l2'}

In [23]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

In [26]:
# make predictions with the trained random forest
test_z = clf_grid.predict(X_test)
test_z_prob = clf_grid.predict_proba(X_test)[:, 1]

In [24]:
# evaluate the model performance - AUC and ROC
accuracy_score(y_test, test_z)

0.9686057248384118

In [27]:
roc_auc_score(y_test, test_z_prob)

0.9734862630034676

Question 4: Use BaggingClassifier to train and evaluate an ensemble model of LogisticRegression  base classifiers. Each base classifier should be trained only on a sample half the size of the training data, and using only half as many features as there are in in total the training data (read the documentation for the function to see how to do this).

In [42]:
from sklearn.ensemble import BaggingClassifier

bagclf = BaggingClassifier(LogisticRegression(solver='liblinear',
                                             C=10,
                                             penalty='l2'), max_samples=.5, max_features=.5)
bagclf.fit(X_train, y_train)

BaggingClassifier(base_estimator=LogisticRegression(C=10, solver='liblinear'),
                  max_features=0.5, max_samples=0.5)

In [43]:
# make predictions with the trained random forest
test_z = bagclf.predict(X_test)
test_z_prob = bagclf.predict_proba(X_test)[:, 1]

In [44]:
# evaluate the model performance - AUC and ROC
accuracy_score(y_test, test_z)

0.96398891966759

In [45]:
roc_auc_score(y_test, test_z_prob)

0.9805148039477194

Question 5: Use AdaBoostClassifier to train and evaluate an ensemble model of LogisticRegression base classifiers.

In [46]:
from sklearn.ensemble import AdaBoostClassifier

adaclf = AdaBoostClassifier(LogisticRegression(solver='liblinear',
                                             C=10,
                                             penalty='l2'))
adaclf.fit(X_train, y_train)

AdaBoostClassifier(base_estimator=LogisticRegression(C=10, solver='liblinear'))

In [47]:
# make predictions with the trained random forest
test_z = adaclf.predict(X_test)
test_z_prob = adaclf.predict_proba(X_test)[:, 1]

In [48]:
# evaluate the model performance - AUC and ROC
accuracy_score(y_test, test_z)

0.9732225300092336

In [49]:
roc_auc_score(y_test, test_z_prob)

0.9780474793278209

[Bonus] Question 6: Use StackingClassifier to train and evaluate an ensemble model of LogisticRegression base classifiers. to get better accuracy than previous classifiers.

In [37]:
from sklearn.ensemble import StackingClassifier

estimators = [
    ('log_l2', LogisticRegression(C=10, solver='liblinear', penalty='l2')),
    ('log_l1', LogisticRegression(C=10, solver='liblinear', penalty='l1')),
    ('log_ada', AdaBoostClassifier(LogisticRegression(solver='liblinear'))),
]
stackclf = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression()
)  
stackclf.fit(X_train, y_train)

StackingClassifier(estimators=[('log_l2',
                                LogisticRegression(C=10, solver='liblinear')),
                               ('log_l1',
                                LogisticRegression(C=10, penalty='l1',
                                                   solver='liblinear')),
                               ('log_ada',
                                AdaBoostClassifier(base_estimator=LogisticRegression(solver='liblinear')))],
                   final_estimator=LogisticRegression())

In [38]:
test_z = stackclf.predict(X_test)
test_z_prob = stackclf.predict_proba(X_test)[:, 1]

In [39]:
accuracy_score(y_test, test_z)

0.9667590027700831

In [40]:
roc_auc_score(y_test, test_z_prob)

0.9776140304081089

Question 7: Create a new text cell in your Notebook: Complete a 50-100 word summary (or short description of your thinking in applying this week's learning to the solution) of your experience in this assignment. Include: What was your incoming experience with this model, if any? what steps you took, what obstacles you encountered. how you link this exercise to real-world, machine learning problem-solving. (What steps were missing? What else do you need to learn?) This summary allows your instructor to know how you are doing and allot points for your effort in thinking and planning, and making connections to real-world work.

I did not have any practical experience with ensemble learning prior to this assignment. I was surprised to see that the bagging classifier was the best method, beating both the base model and the adaboost method. My understanding is that could be a result of a high-variance problem. If the other models are tending to overfit the data, then the bagged model could have higher accuracy because the voting method "smooths" out the prediction function to be less contingent on the particularities of the training data (since each model receives a different slice of the training data). 

In the future, I will consider bagging an ensemble of classifiers if I have a problem with model overfitting. It seems like a very portable solution.